<a href="https://colab.research.google.com/github/WKhisa/Data-Pipelines-with-Python-Project/blob/main/Data_Pipelines_with_Python_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Prerequisites
import pandas as pd
import zipfile 
from zipfile import ZipFile

DATA EXTRACTION

In [3]:
#Read CSV file names within the Zipped File
with ZipFile('dataset1_202302.zip', 'r') as zip:
    zip.printdir()    

File Name                                             Modified             Size
dataset2.csv                                   2023-04-15 10:36:20          738
dataset1.csv                                   2023-04-15 10:36:20          805
dataset3.csv                                   2023-04-15 10:36:20          706


In [4]:
#Create Base FIles
zipped_file = ZipFile('dataset1_202302.zip')
dataset1_df = pd.read_csv(zipped_file.open("dataset1.csv"))
dataset2_df = pd.read_csv(zipped_file.open("dataset2.csv"))
dataset3_df = pd.read_csv(zipped_file.open("dataset3.csv"))

print(dataset1_df.head())
print(dataset2_df.head())
print(dataset3_df.head())



   customer_id date_of_purchase  total_amount_billed payment_status  \
0          101       04/01/2021                  100           paid   
1          102       04/02/2021                  200           paid   
2          103       04/02/2021                   50        overdue   
3          104       04/03/2021                   75       disputed   
4          105       04/04/2021                  125           paid   

  payment_method promo_code country_of_purchase  
0    credit card     PROMO1                 USA  
1  bank transfer     PROMO2                 USA  
2    credit card        NaN                  UK  
3       e-wallet     PROMO3                  UK  
4    credit card     PROMO4                 USA  
   customer_id date_of_payment  amount_paid payment_method payment_status  \
0          101      04/01/2021          100    credit card           paid   
1          102      04/03/2021          200  bank transfer           paid   
2          103      04/03/2021           7

In [57]:
dataset1_df.shape
dataset2_df.shape
dataset3_df.shape

(15, 5)

In [5]:
print(dataset1_df.columns, dataset2_df.columns,dataset3_df.columns )

Index(['customer_id', 'date_of_purchase', 'total_amount_billed',
       'payment_status', 'payment_method', 'promo_code',
       'country_of_purchase'],
      dtype='object') Index(['customer_id', 'date_of_payment', 'amount_paid', 'payment_method',
       'payment_status', 'late_payment_fee', 'country_of_payment'],
      dtype='object') Index(['customer_id', 'date_of_refund', 'refund_amount', 'reason_for_refund',
       'country_of_refund'],
      dtype='object')


TRANSFORMATION
-Join the data sets
-Clean the data set


In [21]:
#Merge DataFrames
merged_dataset_df = pd.merge(dataset1_df, dataset2_df, on = ['customer_id']).merge(dataset3_df, on = ['customer_id'])
merged_dataset_df.head()


,customer_id,date_of_purchase,total_amount_billed,payment_status_x,payment_method_x,promo_code,country_of_purchase,date_of_payment,amount_paid,payment_method_y,payment_status_y,late_payment_fee,country_of_payment,date_of_refund,refund_amount,reason_for_refund,country_of_refund
0,101,04/01/2021,100,paid,credit card,PROMO1,USA,04/01/2021,100,credit card,paid,0,USA,04/03/2021,100,product not as described,USA
1,102,04/02/2021,200,paid,bank transfer,PROMO2,USA,04/03/2021,200,bank transfer,paid,0,USA,04/06/2021,200,defective product,USA
2,103,04/02/2021,50,overdue,credit card,NaN,UK,04/03/2021,75,credit card,paid,10,UK,04/07/2021,75,change of mind,UK
3,104,04/03/2021,75,disputed,e-wallet,PROMO3,UK,04/04/2021,50,e-wallet,overdue,0,UK,04/08/2021,50,product not received,UK
4,105,04/04/2021,125,paid,credit card,PROMO4,USA,04/05/2021,125,credit card,paid,0,USA,04/09/2021,25,product not as described,USA


In [22]:
#Drop Columns not required
merged_dataset_df.drop(columns =['payment_status_x', 'payment_method_x', 'promo_code', 
       'payment_method_y', 'payment_status_y'], inplace = True)
merged_dataset_df.head()

,customer_id,date_of_purchase,total_amount_billed,country_of_purchase,date_of_payment,amount_paid,late_payment_fee,country_of_payment,date_of_refund,refund_amount,reason_for_refund,country_of_refund
0,101,04/01/2021,100,USA,04/01/2021,100,0,USA,04/03/2021,100,product not as described,USA
1,102,04/02/2021,200,USA,04/03/2021,200,0,USA,04/06/2021,200,defective product,USA
2,103,04/02/2021,50,UK,04/03/2021,75,10,UK,04/07/2021,75,change of mind,UK
3,104,04/03/2021,75,UK,04/04/2021,50,0,UK,04/08/2021,50,product not received,UK
4,105,04/04/2021,125,USA,04/05/2021,125,0,USA,04/09/2021,25,product not as described,USA


In [25]:
#check missing values
merged_dataset_df.isna().sum().sum()

0

In [45]:
#Transform Data
merged_dataset_df.dtypes

customer_id              int64
date_of_purchase        object
total_amount_billed    float64
country_of_purchase     object
date_of_payment         object
amount_paid            float64
late_payment_fee       float64
country_of_payment      object
date_of_refund          object
refund_amount          float64
reason_for_refund       object
country_of_refund       object
dtype: object

In [30]:
#convert relevant columns and float data types
merged_dataset_df = merged_dataset_df.astype({'total_amount_billed':'float','amount_paid':'float','late_payment_fee':'float','refund_amount':'float'})
merged_dataset_df.dtypes

customer_id              int64
date_of_purchase        object
total_amount_billed    float64
country_of_purchase     object
date_of_payment         object
amount_paid            float64
late_payment_fee       float64
country_of_payment      object
date_of_refund          object
refund_amount          float64
reason_for_refund       object
country_of_refund       object
dtype: object

In [54]:
#transform date columns
merged_dataset_df[['date_of_purchase', 'date_of_payment', 'date_of_refund']] = merged_dataset_df[['date_of_purchase','date_of_payment', 'date_of_refund']].astype('datetime64[ns]')
merged_dataset_df.dtypes

customer_id                     int64
date_of_purchase       datetime64[ns]
total_amount_billed           float64
country_of_purchase            object
date_of_payment        datetime64[ns]
amount_paid                   float64
late_payment_fee              float64
country_of_payment             object
date_of_refund         datetime64[ns]
refund_amount                 float64
reason_for_refund              object
country_of_refund              object
dtype: object

Load data to CSV for analysis

In [56]:
#save dataframe as excel file
filename= 'final.xlsx'
merged_dataset_df.to_excel(filename)